## 사용 라이브러리

In [1]:
import pandas as pd
import mysql.connector
from datetime import datetime
import time

## 유튜브 데이터 테이블 생성

In [ ]:
# MySQL 데이터베이스 연결 설정
db_config = {
  'user': 'root',
  'password': '12345',
  'host': 'localhost',
  'database': 'test',
}

# CSV 파일에서 데이터 읽기
data = pd.read_csv("D:/1차프로젝트/KR_youtube_trending/KR_youtube_trending_data.csv", encoding='utf-8') 

# 날짜 및 시간 열 변환 및 NaN 값 대체
data['publishedAt'] = pd.to_datetime(data['publishedAt']).dt.strftime('%Y-%m-%d %H:%M:%S')
data['trending_date'] = pd.to_datetime(data['trending_date']).dt.strftime('%Y-%m-%d %H:%M:%S')
data.fillna('', inplace=True)

# 중복된 행 제거 (video_id와 trending_date 기준)
data = data.drop_duplicates(subset=['video_id', 'trending_date'])

# 불필요한 열 삭제
columns_to_drop = ['tags', 'thumbnail_link', 'comments_disabled', 'ratings_disabled', 'description']
data = data.drop(columns=columns_to_drop)

try:
    # MySQL DB 연결
    conn = mysql.connector.connect(**db_config)

    # Cursor 객체 생성
    cursor = conn.cursor()
    
    # 시작 시간 기록
    start_time = time.time()
    
    # 기존 테이블 삭제
    cursor.execute("DROP TABLE IF EXISTS youtube_trending")
    
    # 새 테이블 생성
    cursor.execute("""
        CREATE TABLE youtube_trending (
            video_id VARCHAR(255),
            title TEXT,
            publishedAt DATETIME,
            publishedAt_year INT,
            publishedAt_month INT,
            publishedAt_day INT,
            publishedAt_weekday INT,
            channelId VARCHAR(255),
            channelTitle TEXT,
            categoryId INT,
            trending_date DATETIME,
            trending_date_year INT,
            trending_date_month INT,
            trending_date_day INT,
            trending_date_weekday INT,
            view_count BIGINT,
            likes BIGINT,
            dislikes BIGINT,
            comment_count BIGINT,
            PRIMARY KEY (video_id, trending_date)
        )
    """)

    # 유튜브 트랜드 데이터 삽입 쿼리
    insert_query = "INSERT INTO youtube_trending (video_id, title, publishedAt, channelId, channelTitle, categoryId, trending_date, view_count, likes, dislikes, comment_count) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"

    # 유튜브 트랜드 데이터 삽입
    for index, row in data.iterrows():
        cursor.execute(insert_query, (row['video_id'], row['title'], row['publishedAt'], row['channelId'], row['channelTitle'], row['categoryId'], row['trending_date'], row['view_count'], row['likes'], row['dislikes'], row['comment_count']))

    # 변경사항 저장
    conn.commit()

    # 'publishedAt'와 'trending_date' 열 분리 및 업데이트
    update_query = """
        UPDATE youtube_trending
        SET
            publishedAt_year = YEAR(publishedAt),
            publishedAt_month = MONTH(publishedAt),
            publishedAt_day = DAY(publishedAt),
            publishedAt_weekday = DAYOFWEEK(publishedAt),
            trending_date_year = YEAR(trending_date),
            trending_date_month = MONTH(trending_date),
            trending_date_day = DAY(trending_date),
            trending_date_weekday = DAYOFWEEK(trending_date)
    """
    cursor.execute(update_query)
    conn.commit()

    # 종료 시간 기록
    end_time = time.time()

    # 실행 시간 계산
    elapsed_time = end_time - start_time
    print(f"코드 실행 시간: {round(elapsed_time, 3)} 초")
    
except mysql.connector.Error as err:
    print("Something went wrong: {}".format(err))
    
finally:
    if (conn.is_connected()):
        # 연결 닫기
        cursor.close()
        conn.close()